In [25]:
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from transformers import TFBertForSequenceClassification, BertTokenizerFast

In [22]:
df = pd.read_csv(r"C:\Users\ASatl\OneDrive\Desktop\let\tweets.csv")
df

,No.,Message
0,1,Excited to see what's next for $AAPL in the te...
1,2,$AAPL's stock performance reflects its solid f...
2,3,Wondering how $AAPL will capitalize on the lat...
3,4,$AAPL's innovation engine never ceases to amaze.
4,5,Anticipating the impact of $AAPL's upcoming pr...
...,...,...
183,184,Investors remain bullish on $AAPL despite regu...
184,185,The success of Apple Fitness+ highlights $AAPL...
185,186,$AAPL's commitment to reducing its environment...
186,187,Investors are excited about $AAPL's expansion ...


In [6]:
# Load pre-trained FinBERT model and tokenizer
finbert_model = TFBertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)
finbert_tokenizer = BertTokenizerFast.from_pretrained('yiyanghkust/finbert-tone')

c:\Users\ASatl\OneDrive\Desktop\let\venv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASatl\.cache\huggingface\hub\models--yiyanghkust--finbert-tone. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at yiyanghkust/finbert-tone.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [20]:
# Extract tweet texts from the "Message" column
tweet_texts = df["Message"].tolist()

# Tokenize each tweet text into individual words
tweet_tokens = [tweet_text.split() for tweet_text in tweet_texts]

# Train Word2Vec model on the tokenized tweet data
word2vec_model = Word2Vec(tweet_tokens, vector_size=100, window=5, min_count=1, workers=4)


In [23]:
df_stock = pd.read_csv(r"C:\Users\ASatl\OneDrive\Desktop\let\AAPL.csv")
df_stock

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-01-03,0.936384,1.004464,0.907924,0.999442,0.847207,535796800
1,2000-01-04,0.966518,0.987723,0.903460,0.915179,0.775779,512377600
2,2000-01-05,0.926339,0.987165,0.919643,0.928571,0.787131,778321600
3,2000-01-06,0.947545,0.955357,0.848214,0.848214,0.719014,767972800
4,2000-01-07,0.861607,0.901786,0.852679,0.888393,0.753073,460734400
...,...,...,...,...,...,...,...
6041,2024-01-08,182.089996,185.600006,181.500000,185.559998,185.559998,59144500
6042,2024-01-09,183.919998,185.149994,182.729996,185.139999,185.139999,42841800
6043,2024-01-10,184.350006,186.399994,183.919998,186.190002,186.190002,46792900
6044,2024-01-11,186.539993,187.050003,183.619995,185.589996,185.589996,49128400


In [26]:
# Concatenate Word2Vec embeddings
tweet_embeddings = []
for tweet_text in tweet_texts:
    tweet_embedding = [word2vec_model.wv[word] for word in tweet_text.split() if word in word2vec_model.wv]
    if tweet_embedding:
        tweet_embedding_avg = np.mean(tweet_embedding, axis=0)
        tweet_embeddings.append(tweet_embedding_avg)

In [30]:
# Concatenate features and target
X = np.array(tweet_embeddings)
y = df_stock['Close'].values[:len(tweet_embeddings)]  # Use only as many stock prices as the number of tweets

In [31]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
# Initialize MLP Regressor model
mlp_regressor = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)

# Train MLP Regressor model
mlp_regressor.fit(X_train, y_train)

# Predict stock prices
predictions = mlp_regressor.predict(X_test)

In [33]:
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.01515188934633999
